In [ ]:
import speech_recognition as sr
from gtts import gTTS
from os import remove, path
import pyglet
import re
import datetime
import requests
import smtplib

In [ ]:
class todoList:
    def __init__(self, filename):
        self.filename = filename
        if not path.exists(self.filename):
            with open(self.filename, 'w') as file:
                file.close()

    def addTask(self, task):
        with open(self.filename, 'a') as file:
            file.write(task+'\n')
        return 'Task added successfully!'

    def listTasks(self):
        with open(self.filename, 'r') as file:
            tasks = file.readlines()
            tasklist = ''
            for index, task in enumerate(tasks, start = 1):
                tasklist += 'Number ' + str(index) + ': ' + task[:-1] + '. '
            return f'You have {len(tasks)} tasks in your To Do List currently. {tasklist}'

    def removeTask(self, task):
        updated= []
        found = 0
        with open(self.filename, 'r') as file:
            tasks = file.readlines()
            for rec in tasks:
                if rec[:-1] != task:
                    updated.append(rec)
                else:
                    found = 1
        if not found:
            return 'No such task exists!'
        with open(self.filename, 'w+') as file:
            file.writelines(updated)
            file.seek(0)
            return f'Task has been removed successfully! You now have {len(file.readlines())} tasks in your list.'

In [ ]:
class codeMapper:
    def __init__(self):
        self.code_map = {(0,): 'We have a clear sky',
                         (1,): 'It is mainly clear',
                         (2,): 'It is partly cloudy',
                         (3,): 'It is overcast',
                         (45, 48): 'It is foggy',
                         (52, 53): 'There is a moderate drizzle',
                         (55, 56, 57): 'There is a dense, freezing drizzle',
                         (61, 63): 'It is raining',
                         (65, 66, 67) : 'There is a dense, freezing rain out',
                         (71, 73) : 'There is a moderate snow fall',
                         (75, 77) : 'There is an intense snow fall',
                         (80, 81, 82) : 'There is a rain shower',
                         (85, 86) : 'There is a snow shower',
                         (95, 96, 99) : 'There is a thunderstorm'
                        }

In [ ]:
class mailer:

    def __init__(self, sender_email, sender_password, server = "smtp.gmail.com", port = 465):
        self.server = server
        self.port = port
        self.sender_email = sender_email
        self.sender_password = sender_password

    def setRecepient(self):
        self.recepient_email = input('Enter recepient mail id: ')

    def sendMail(self, message):
        try:
            with smtplib.SMTP_SSL(self.server, self.port) as server:
                server.login(self.sender_email, self.sender_password)
                server.sendmail(self.sender_email, self.recepient_email, message)
        except smtplib.SMTPAuthenticationError:
            return 'Couldn\'t send mail. Please check your credentials and Google Account Settings'
        except:
            return 'Couldn\'t send mail. Facing some issues.'
        return 'Mail Sent Successfully'         

In [ ]:
class assistant:

    def __init__(self, wakeWord):
        
        self.wakeWord = wakeWord.lower()
        
        self.ipinfoToken = r"""ENTER YOUR FREE AUTH TOKEN FROM IPINFO WEBSITE HERE"""
        self.code_map = codeMapper().code_map
        
        self.todolist = todoList(f'{self.wakeWord} Task List')
        self.taskListen = 0

        self.mailer = mailer(r"""ENTER YOUR GMAIL ADDR HERE""", r"""ENTER YOUR GMAIL PASSWORD HERE""")
        self.mailListen = 0
    
    def listener(self):
    
        recognizer = sr.Recognizer()
    
        with sr.Microphone() as mic:
            recognizer.adjust_for_ambient_noise(mic)
            audio = recognizer.listen(mic)
    
        try:
            command = recognizer.recognize_google(audio)
            return 0, command.lower().strip()
        except sr.UnknownValueError:
            return 1, "Could not grasp what you said. Please repeat yourself."      
        except sr.RequestError:
            return 2, "Unable to access the Google Speech Recognition API."


    def responder(self, response):
        
        tts = gTTS(text = response, lang='en')
        tts.save("response.mp3")
        
        pyglet.media.load("response.mp3").play()
        
        remove("response.mp3")

    def run(self):

        while True:

            status, msg = self.listener()
            response = ''

            if status == 2:
                self.responder(msg)
                break

            elif status == 1:
                response = msg

            else:
                
                print("You said:", repr(msg))

                if self.taskListen:
                    if self.taskListen == 1:
                        response = self.todolist.addTask(msg)
                    else:
                        response = self.todolist.removeTask(msg)
                    self.taskListen = 0  

                elif self.mailListen:
                    response = self.mailer.sendMail(msg)
                    self.mailListen = 0
                
                elif re.match('^hey|hay chip', msg):

                    if msg == "hey chip":
                        response = "Hello! How may I help you today?"

                    elif 'date' in msg:
                        
                        if 'today' in msg:
                            response = 'Today\'s date is ' + str(datetime.date.today())

                        elif 'tomorrow' in msg:
                            response = 'Tomorrow\'s date is' + (datetime.datetime.now() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')

                        elif 'yesterday' in msg:
                            response = 'Yesterday\'s date was' + (datetime.datetime.now() + datetime.timedelta(days=-1)).strftime('%Y-%m-%d')

                    elif 'time' in msg:
                        
                        time = datetime.datetime.now().strftime('%H:%M')
                        suf = 'am' if int(time[:time.find(':')]) < 12 else 'pm'
                        response = 'It is currently ' + time + ' ' + suf

                    elif 'weather' in msg or 'temperature' in msg:
                        
                        loc = requests.get(url = f'https://ipinfo.io?token={self.ipinfoToken}').json()['loc'].split(',')
                        loc = list(map(float, loc))
                        
                        weather = requests.get(url = f"https://api.open-meteo.com/v1/forecast?latitude={loc[0]}&longitude={loc[1]}&current=temperature_2m&current=weather_code").json()
                        temp = weather['current']['temperature_2m']
                        code = weather['current']['weather_code']
                        
                        key = [x for x in self.code_map.keys() if code in x][0]
                        desc = self.code_map.get(key, 'I am unsure about the weather ')
                        response = desc + ' today. The current temperature is ' + str(temp) + ' degree celsius.'
                    
                    elif 'task' in msg:
                        
                        if  'list' in msg or 'what' in msg:
                            response = self.todolist.listTasks()

                        elif 'add' in msg or 'make' in msg:
                            response = 'Sure! What task do you want to add?'
                            self.taskListen = 1

                        elif 'remove' in msg or 'delete' in msg:
                            response = 'Sure! What task do you want to remove?'
                            self.taskListen = 2
                    
                    elif 'mail' in msg and ('send' in msg or 'write' in msg):
                        self.responder('Sure! Who do you want to send the mail to?')
                        self.mailer.setRecepient()
                        response = 'Recepient added successfully! What\'s your message?'
                        self.mailListen = 1
                        
                    elif 'stop' in msg:
                        self.responder('Bye Bye!')
                        break

                    else:
                        response = 'Sorry... I don\'t know how to do that!'

                if response:
                    self.responder(response)


In [ ]:
if __name__ == '__main__':
    VoiceAssistant = assistant('Chip')
    VoiceAssistant.run()